In [10]:
import numpy as np
# importing decimal module to control significant figures
import decimal

In [11]:
def toDecimal(array) :
    # flatten the array convert floats to decimal and reshape to original shape
    originalShape = array.shape
    array = array.flatten().astype(object) 
    for i in range(array.size) :
        array[i] = decimal.Decimal(str(array[i]))
    array = array.reshape(originalShape)   
    return array


def intializeContext(significantFigs,rounding) :
    #Intialize the context with specified signFigs and rounding/chopping
    decimal.getcontext().prec = significantFigs 
    if( rounding ) :
        decimal.getcontext().rounding = decimal.ROUND_HALF_UP
    else :
        decimal.getcontext().rounding = decimal.ROUND_DOWN 
        
def toFloats (array) :
    #Convert the decimals back to float
    originalShape = array.shape
    array = array.flatten().astype(object)
    for i in range(array.size) :
        array[i] = float(array[i]) 
    array = array.reshape(originalShape)    
    return array            
    

In [12]:
def isDiagonalyDominant(A) :
    #Check if Matrix A is diagonaly dominant 
    dim = A.shape
    # Flags to check system
    at_least_one_strictly_greater = None 
    if(dim[0] != dim[1]) : raise Exception("Matrix must be square")
    for i in range(dim[0]) :
        nonDiagonalSum = 0 
        for j in range(dim[0]) :
            if(j==i) : continue 
            nonDiagonalSum += abs(A[i][j]) 
        if(A[i][i] > nonDiagonalSum) :
            at_least_one_strictly_greater = True
        elif(A[i][i] == nonDiagonalSum) :
            if(at_least_one_strictly_greater == None) : at_least_one_strictly_greater = False
        else :
            return False    
    return at_least_one_strictly_greater            
            

In [13]:
# Jacobi with relaxation for improved convergence (called weighted Jacobi)
# the parameter n can be removed as we can get the no of vars from the npArray
def Jacobi_noNorm(A,b,n,x,maxIterations,ErrorTolerance,relax , significantFigs = 7 , rounding = True) :
    # Setting up signifcant figs and rounding/chopping
    intializeContext(significantFigs,rounding)
    #Copying the arrays to avoid modifying original arrays
    A = A.copy()
    b = b.copy()
    x = x.copy()
    #converting floats to decimals 
    A = toDecimal(A)
    b = toDecimal(b)
    x = toDecimal(x)
    relax = decimal.Decimal(str(relax))
    #Array to hold new values
    xNew = np.zeros(n,dtype=object)
    #List to hold iteration details
    iteration_details = []
    if(isDiagonalyDominant(A) ) :
        isDominant = 'The matrix is diagonaly dominant'
    else :
        isDominant = 'The matrix is not diagonaly dominant' 
    # Calculating first iteration before applying relaxation    
    for i in range(n) :
        sum = b[i]
        for j in range(n) :
            if(i==j) :
                continue
            sum -= A[i][j] * x[j]
        xNew[i] = sum/A[i][i]
    x = xNew.copy()
     # Storing details of first iteration    
    details = {
            'iteration' : 1,
            'xNew' : toFloats(xNew),
            'maxError' : '_'
        }     
    iteration_details.append(details)
    iteration = 2
    # Loop until convergence or max iterations reached 
    while (True) :
        belowTolerance = True
        maxError = decimal.Decimal('0')
        for i in range(n) :
            oldX = x[i]
            sum = b[i]
            for j in range(n) :
                if(i==j) :
                    continue
                sum -= A[i][j] * x[j]
            xNew[i] = relax*sum/A[i][i] + (1-relax)*oldX
            if (xNew[i] != 0) :
                estimatedError = abs((xNew[i]-oldX)/xNew[i]) * 100
                maxError = max(maxError, estimatedError)
                if(estimatedError > ErrorTolerance):
                    belowTolerance = False
        details = {
            'iteration' : iteration,
            'xNew' : toFloats(xNew),
            'maxError' : float(maxError)
        }         
        iteration_details.append(details)
        iteration+=1
        x = xNew.copy()
        if(belowTolerance or iteration >= maxIterations) :
            break
    lines = [isDominant]
    for d in iteration_details :
        s = (
            f"Iteration: {d['iteration']}\n"
            f"  xNew: {d['xNew']}\n"
            f"  maxError: {d['maxError']}"
        )
        lines.append(s)        
    return toFloats(xNew) , lines

In [14]:
# =========================
# Test the Jacobi_noNorm function
# =========================

# Example linear system
A = np.array([[10, -1, 2, 0],
              [-1, 11, -1, 3],
              [2, -1, 10, -1],
              [0, 3, -1, 8]], dtype=float)

b = np.array([6, 25, -11, 15], dtype=float)

# Initial guess
x0 = np.zeros(len(b))

# Run weighted Jacobi
solution, details = Jacobi_noNorm(A, b, 4, x0, maxIterations=50, ErrorTolerance=1e-6, relax=0.8)

print("Solution:", solution)
print("\nIteration Details:")
print("\n".join(details))
# for d in details:
#     max_error = d['maxError']
#     # Ensure maxError is a float, else print as-is
#     if isinstance(max_error, (float, int)):
#         max_error_str = f"{max_error:.6f}"
#     else:
#         max_error_str = str(max_error)
#     print(f"Iteration {d['iteration']}: x = {d['xNew']}, maxError = {max_error_str}")


Solution: [1.0 2.0 -1.0 1.0]

Iteration Details:
The matrix is diagonaly dominant
Iteration: 1
  xNew: [0.6 2.272727 -1.1 1.875]
  maxError: _
Iteration: 2
  xNew: [0.9578184 1.827272 -0.8641818 1.083182]
  maxError: 73.1011
Iteration: 3
  xNew: [0.9560146 1.954115 -0.9732516 1.082037]
  maxError: 11.20674
Iteration: 4
  xNew: [0.9832523 1.97167 -0.9847209 1.032848]
  maxError: 4.762463
Iteration: 5
  xNew: [0.9919395 1.98706 -0.9939036 1.016597]
  maxError: 1.598569
Iteration: 6
  xNew: [0.9963773 1.993648 -0.9971985 1.007811]
  maxError: 0.8717904
Iteration: 7
  xNew: [0.9983191 1.996966 -0.9987436 1.003748]
  maxError: 0.4047829
Iteration: 8
  xNew: [0.99922 1.998545 -0.9994225 1.001785]
  maxError: 0.1959502
Iteration: 9
  xNew: [0.9996352 1.999305 -0.9997337 1.000851]
  maxError: 0.09332058
Iteration: 10
  xNew: [0.9998288 1.999669 -0.9998762 1.000405]
  maxError: 0.04458194
Iteration: 11
  xNew: [0.9999195 1.999842 -0.999942 1.000193]
  maxError: 0.02119591
Iteration: 12
  xNew: 

In [15]:
A = np.array([[3, -0.1, -0.2],[0.1,7,-0.3], [0.3,-0.2,10]], dtype=float)
b = np.array([7.85, -19.3, 71.4], dtype=float)
solution, details = Jacobi_noNorm(A, b, 3, np.zeros(3), maxIterations=25, ErrorTolerance=1e-6, relax=1.0)
print("Solution:", solution)
print("\nIteration Details:")
print("\n".join(details))

Solution: [3.0 -2.5 7.0]

Iteration Details:
The matrix is diagonaly dominant
Iteration: 1
  xNew: [2.616667 -2.757143 7.14]
  maxError: _
Iteration: 2
  xNew: [3.000762 -2.488524 7.006357]
  maxError: 12.79992
Iteration: 3
  xNew: [3.000806 -2.499739 7.000207]
  maxError: 0.4486468
Iteration: 4
  xNew: [3.000022 -2.500003 6.999981]
  maxError: 0.02613314
Iteration: 5
  xNew: [2.999999 -2.500001 6.999999]
  maxError: 0.0007666669
Iteration: 6
  xNew: [3.0 -2.5 7.0]
  maxError: 4e-05
Iteration: 7
  xNew: [3.0 -2.5 7.0]
  maxError: 0.0


In [16]:
A = np.array(([5,-1,1],[2,8,-1],[-1,1,4]), dtype=float)
b = np.array([10,11,3], dtype=float)
solution, details = Jacobi_noNorm(A, b, 3, np.zeros(3), maxIterations=25, ErrorTolerance=1e-6, relax=1.0)
print("Solution:", solution)    
print("\nIteration Details:")
print("\n".join(details))

Solution: [2.0 1.0 1.0]

Iteration Details:
The matrix is diagonaly dominant
Iteration: 1
  xNew: [2.0 1.375 0.75]
  maxError: _
Iteration: 2
  xNew: [2.125 0.96875 0.90625]
  maxError: 41.93548
Iteration: 3
  xNew: [2.0125 0.9570313 1.039063]
  maxError: 12.782
Iteration: 4
  xNew: [1.983593 1.001758 1.013867]
  maxError: 4.464821
Iteration: 5
  xNew: [1.997579 1.005835 0.9954588]
  maxError: 1.849218
Iteration: 6
  xNew: [2.002076 1.000038 0.997936]
  maxError: 0.579678
Iteration: 7
  xNew: [2.00042 0.999223 1.00051]
  maxError: 0.2572688
Iteration: 8
  xNew: [1.999742 0.9999588 1.000299]
  maxError: 0.07358303
Iteration: 9
  xNew: [1.999932 1.000102 0.9999458]
  maxError: 0.03532191
Iteration: 10
  xNew: [2.00003 1.00001 0.9999575]
  maxError: 0.009199908
Iteration: 11
  xNew: [2.00001 0.9999873 1.000005]
  maxError: 0.004749976
Iteration: 12
  xNew: [1.999997 0.9999981 1.000006]
  maxError: 0.001080002
Iteration: 13
  xNew: [1.999999 1.000002 0.9999998]
  maxError: 0.0006200001
Ite